In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import zip_longest
import os
import glob

In [3]:
input_folder = "Manual measurements inperson"
output_folder = "Manual measurements inperson cleaned"
BATCH_SIZE = 5000  # files (measurements) per flush

os.makedirs(output_folder, exist_ok=True)  # ensure folder exists
final_csv = os.path.join(output_folder, "Manual_measurements_cleaned.csv")  # <-- final file name

files = glob.glob(os.path.join(input_folder, "*.json"))

run_counter = 0
wrote_header = False
batch = []
columns_order = None  # will lock on first batch write
accepted_in_batch = 0

for file in files:
    try:
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        df = pd.json_normalize(
            data,
            record_path="raw_measurements",
            meta=["version","type","start_time","end_time","sampling_rate",
                  "weight_filter","zero_offset","expected_weight","robot_type","force_sensor_offset"],
            errors="ignore"
        )

        df["timestamp"] = df["timestamp"] - df["timestamp"].iloc[0]

        # your checks
        if len(df) < 400:               continue
        if "value" not in df.columns:   continue
        if df["value"].isna().all() or (df["value"] == 0).all():  continue

        # ids
        df["measurement_id"] = os.path.splitext(os.path.basename(file))[0]
        run_counter += 1
        df["run_index"] = run_counter

        batch.append(df)
        accepted_in_batch += 1

        # flush every BATCH_SIZE accepted files
        if accepted_in_batch >= BATCH_SIZE:
            out = pd.concat(batch, ignore_index=True)

            # lock column order on first write; enforce consistently thereafter
            if columns_order is None:
                columns_order = out.columns.tolist()
            else:
                out = out.reindex(columns=columns_order, fill_value=pd.NA)

            out.to_csv(final_csv, mode="a", index=False, encoding="utf-8",
                       header=not wrote_header)
            wrote_header = True

            # clear memory
            batch.clear()
            accepted_in_batch = 0
            del out
            gc.collect()

    except Exception:
        continue

# flush remainder
if batch:
    out = pd.concat(batch, ignore_index=True)
    if columns_order is None:
        columns_order = out.columns.tolist()
    else:
        out = out.reindex(columns=columns_order, fill_value=pd.NA)
    out.to_csv(final_csv, mode="a", index=False, encoding="utf-8",
               header=not wrote_header)

print(f"Done. Appended to {os.path.abspath(final_csv)}")

Done. Appended to C:\Users\piete\Documents\Smart Robotics PROJECT\Manual measurements inperson cleaned\all_measurements_correct.csv


In [11]:
df = pd.read_csv("Manual measurements inperson cleaned/Manual_measurements_cleaned.csv")

In [12]:
df

,timestamp,force_vector,value,robot_tcp.timestamp,robot_tcp.tcp_offset,robot_tcp.joint_angles,robot_tcp.velocity_angular,robot_tcp.velocity_linear,robot_tcp.flange,version,...,start_time,end_time,sampling_rate,weight_filter,zero_offset,expected_weight,robot_type,force_sensor_offset,measurement_id,run_index
0,0.000000,"[-0.81, -3.294, 25166.0]",0.249,1.759831e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-1.561415974293844, -1.7694484196104945, -1.8...","[-0.0011338490123100062, 0.002360197018098688,...","[-0.052403629897558555, 0.10364571818892869, 1...","[[-0.16906002163887024, -0.751794159412384, 0....",2,...,1.759831e+09,1.759831e+09,1000.0,NaN,0.133159,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_11-53-59-574052_measure_weight_FORC...,1
1,0.002102,"[-0.828, -3.354, 27135.0]",0.245,1.759831e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-1.5649998823748987, -1.7314983807005824, -1....","[0.01950379755510171, 0.007591311535297083, -0...","[-0.1104457979607352, 0.22416212288660517, 1.2...","[[-0.16930893063545227, -0.7513507604598999, 0...",2,...,1.759831e+09,1.759831e+09,1000.0,NaN,0.133159,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_11-53-59-574052_measure_weight_FORC...,1
2,0.002830,"[-0.845, -3.379, 27161.0]",0.243,1.759831e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.1695263683795929, -0.7509328722953796, 0....",2,...,1.759831e+09,1.759831e+09,1000.0,NaN,0.133159,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_11-53-59-574052_measure_weight_FORC...,1
3,0.003806,"[-0.869, -3.401, 26560.0]",0.242,1.759831e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.1695263683795929, -0.7509328722953796, 0....",2,...,1.759831e+09,1.759831e+09,1000.0,NaN,0.133159,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_11-53-59-574052_measure_weight_FORC...,1
4,0.004807,"[-0.897, -3.418, 25701.0]",0.241,1.759831e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.16977432370185852, -0.7504664659500122, 0...",2,...,1.759831e+09,1.759831e+09,1000.0,NaN,0.133159,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_11-53-59-574052_measure_weight_FORC...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90345,0.496445,"[-0.158, -0.076, 29409.0]",0.258,1.759844e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.5975596904754639, -0.1040428876876831, 0....",2,...,1.759844e+09,1.759844e+09,1000.0,NaN,-0.087844,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_15-26-06-735413_measure_weight_FORC...,122
90346,0.497445,"[-0.169, -0.076, 29394.0]",0.258,1.759844e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.5975596904754639, -0.1040428876876831, 0....",2,...,1.759844e+09,1.759844e+09,1000.0,NaN,-0.087844,0.165,ur10e,"[[0.0, 0.0, 0.02], [1.0, 0.0, 0.0, 0.0]]",2025-10-07_15-26-06-735413_measure_weight_FORC...,122
90347,0.498445,"[-0.181, -0.077, 29414.0]",0.258,1.759844e+09,"[[0.0, 0.0, 0.41], [1.0, 0.0, 0.0, 0.0]]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.597572922706604, -0.10404746234416962, 0....",2,

In [13]:
df = df.drop('start_time', axis=1)
df = df.drop('end_time', axis=1)
df = df.drop('version', axis=1)
df = df.drop('timestamp', axis=1)
df = df.drop('type', axis=1)
df = df.drop('sampling_rate', axis=1)
df = df.drop('weight_filter', axis=1)
df = df.drop('robot_type', axis=1)
df = df.drop('measurement_id', axis=1)
df = df.drop('robot_tcp.timestamp', axis=1)
df = df.drop('force_sensor_offset', axis=1) # always 1, so contains no information.
df = df.drop('robot_tcp.tcp_offset', axis=1) # always 1, so contains no information.

df['force_sensor_value'] = df['value'] - df['zero_offset'] # does the zero_offset only impact the 'value' colummn? My assumption is yes, but not 100% sure.
df = df.drop('value', axis=1)
df = df.drop('zero_offset', axis=1)

In [14]:
df.keys()

Index(['force_vector', 'robot_tcp.joint_angles', 'robot_tcp.velocity_angular',
       'robot_tcp.velocity_linear', 'robot_tcp.flange', 'expected_weight',
       'run_index', 'force_sensor_value'],
      dtype='object')

In [15]:
df

,force_vector,robot_tcp.joint_angles,robot_tcp.velocity_angular,robot_tcp.velocity_linear,robot_tcp.flange,expected_weight,run_index,force_sensor_value
0,"[-0.81, -3.294, 25166.0]","[-1.561415974293844, -1.7694484196104945, -1.8...","[-0.0011338490123100062, 0.002360197018098688,...","[-0.052403629897558555, 0.10364571818892869, 1...","[[-0.16906002163887024, -0.751794159412384, 0....",0.165,1,0.115841
1,"[-0.828, -3.354, 27135.0]","[-1.5649998823748987, -1.7314983807005824, -1....","[0.01950379755510171, 0.007591311535297083, -0...","[-0.1104457979607352, 0.22416212288660517, 1.2...","[[-0.16930893063545227, -0.7513507604598999, 0...",0.165,1,0.111841
2,"[-0.845, -3.379, 27161.0]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.109841
3,"[-0.869, -3.401, 26560.0]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.108841
4,"[-0.897, -3.418, 25701.0]","[-1.5652936140643519, -1.7292896709837855, -1....","[0.023282260836118857, 0.007685194040913373, -...","[-0.11403168777029214, 0.23437901113110501, 1....","[[-0.16977432370185852, -0.7504664659500122, 0...",0.165,1,0.107841
...,...,...,...,...,...,...,...,...
90345,"[-0.158, -0.076, 29409.0]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844
90346,"[-0.169, -0.076, 29394.0]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844
90347,"[-0.181, -0.077, 29414.0]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844
90348,"[-0.194, -0.078, 29422.0]","[-2.6771720091449183, -1.5029736843756218, -1....","[-4.291573843208506e-11, 8.582081738676044e-11...","[-2.1818497998697004e-11, -1.0905309228259302e...","[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844


In [16]:
for col in df.columns:
    s = df[col]
    if s.dtype == object and s.str.startswith('[').all() and ~s.str.contains(r'\[\[').all():
        parts = s.str[1:-1].str.split(',', expand=True)        # vectorized split
        parts = parts.apply(pd.to_numeric, errors='coerce')     # vectorized cast
        parts.columns = [f"{col}_{i+1}" for i in range(parts.shape[1])]
        df[parts.columns] = parts
        df.drop(columns=[col], inplace=True)

In [17]:
df

,robot_tcp.flange,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.joint_angles_1,robot_tcp.joint_angles_2,robot_tcp.joint_angles_3,robot_tcp.joint_angles_4,robot_tcp.joint_angles_5,robot_tcp.joint_angles_6,robot_tcp.velocity_angular_1,robot_tcp.velocity_angular_2,robot_tcp.velocity_angular_3,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3
0,"[[-0.16906002163887024, -0.751794159412384, 0....",0.165,1,0.115841,-0.810,-3.294,25166.0,-1.561416,-1.769448,-1.829999,-1.088762,1.580291,-0.988507,-1.133849e-03,2.360197e-03,-3.150696e-04,-5.240363e-02,1.036457e-01,1.326491e+00
1,"[[-0.16930893063545227, -0.7513507604598999, 0...",0.165,1,0.111841,-0.828,-3.354,27135.0,-1.565000,-1.731498,-1.790243,-1.166664,1.580008,-0.991539,1.950380e-02,7.591312e-03,-9.518018e-04,-1.104458e-01,2.241621e-01,1.293527e+00
2,"[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.109841,-0.845,-3.379,27161.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
3,"[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.108841,-0.869,-3.401,26560.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
4,"[[-0.16977432370185852, -0.7504664659500122, 0...",0.165,1,0.107841,-0.897,-3.418,25701.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90345,"[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844,-0.158,-0.076,29409.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90346,"[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844,-0.169,-0.076,29394.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90347,"[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844,-0.181,-0.077,29414.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90348,"[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844,-0.194,-0.078,29422.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11


In [18]:
col = 'robot_tcp.flange'

s = df[col]

# Clean like before (flatten and remove brackets/spaces)
inner = (
    s.astype(str)
     .str.replace('],', ',', regex=False)
     .str.replace('[', '', regex=False)
     .str.replace(']', '', regex=False)
     .str.replace(' ', '', regex=False)
     .str.replace('\t', '', regex=False)
     .str.strip(',')
)

# Parse each row individually and count numeric tokens
counts = inner.map(lambda x: len(np.fromstring(x, sep=',')))
commas = inner.str.count(',')
expected = commas.mode()[0] + 1 if not commas.mode().empty else None

# Find bad rows
bad_mask = (expected is not None) & ((counts != expected) | (counts == 0))
bad_rows = df.index[bad_mask].tolist()

if bad_rows:
    print(f"⚠️ Found {len(bad_rows)} malformed rows in '{col}' (expected {expected} numbers per row):\n")
    for i in bad_rows[:10]:  # show up to 10 samples
        print(f"Row {i}: {df.at[i, col]}")
else:
    print(f"✅ All rows in '{col}' parsed cleanly ({expected} numbers per row).")

✅ All rows in 'robot_tcp.flange' parsed cleanly (7 numbers per row).


In [19]:
df

,robot_tcp.flange,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.joint_angles_1,robot_tcp.joint_angles_2,robot_tcp.joint_angles_3,robot_tcp.joint_angles_4,robot_tcp.joint_angles_5,robot_tcp.joint_angles_6,robot_tcp.velocity_angular_1,robot_tcp.velocity_angular_2,robot_tcp.velocity_angular_3,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3
0,"[[-0.16906002163887024, -0.751794159412384, 0....",0.165,1,0.115841,-0.810,-3.294,25166.0,-1.561416,-1.769448,-1.829999,-1.088762,1.580291,-0.988507,-1.133849e-03,2.360197e-03,-3.150696e-04,-5.240363e-02,1.036457e-01,1.326491e+00
1,"[[-0.16930893063545227, -0.7513507604598999, 0...",0.165,1,0.111841,-0.828,-3.354,27135.0,-1.565000,-1.731498,-1.790243,-1.166664,1.580008,-0.991539,1.950380e-02,7.591312e-03,-9.518018e-04,-1.104458e-01,2.241621e-01,1.293527e+00
2,"[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.109841,-0.845,-3.379,27161.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
3,"[[-0.1695263683795929, -0.7509328722953796, 0....",0.165,1,0.108841,-0.869,-3.401,26560.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
4,"[[-0.16977432370185852, -0.7504664659500122, 0...",0.165,1,0.107841,-0.897,-3.418,25701.0,-1.565294,-1.729290,-1.787980,-1.170969,1.579972,-0.991756,2.328226e-02,7.685194e-03,-1.365607e-03,-1.140317e-01,2.343790e-01,1.289273e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90345,"[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844,-0.158,-0.076,29409.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90346,"[[-0.5975596904754639, -0.1040428876876831, 0....",0.165,122,0.345844,-0.169,-0.076,29394.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90347,"[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844,-0.181,-0.077,29414.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11
90348,"[[-0.597572922706604, -0.10404746234416962, 0....",0.165,122,0.345844,-0.194,-0.078,29422.0,-2.677172,-1.502974,-1.141248,-2.068237,1.564085,-0.156833,-4.291574e-11,8.582082e-11,-7.608977e-15,-2.181850e-11,-1.090531e-11,5.971910e-11


In [20]:
col = "robot_tcp.flange"
chunk = 1_000_000

s = (df[col].astype(str)
       .str.replace(r'\],\s*\[', ',', regex=True)
       .str.replace(r'[\[\]\s]', '', regex=True)
       .str.replace(',,', ',', regex=False)
       .str.strip(','))

w = s.iloc[0].count(',') + 1
out = np.empty((len(df), w), float)

for start in range(0, len(df), chunk):
    end = min(start + chunk, len(df))
    block = s.iloc[start:end].to_numpy(copy=False)
    big = '\n'.join(block.tolist())
    arr = np.fromstring(big, sep=',', dtype=float)
    if arr.size != (end - start) * w:
        arr = np.concatenate([np.fromstring(x, sep=',', dtype=float) for x in block])
    out[start:end] = arr.reshape(end - start, w)

df[[f"{col}_{i+1}" for i in range(w)]] = out
df.drop(columns=[col], inplace=True)

C:\Users\piete\AppData\Local\Temp\ipykernel_9608\787671431.py:17: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  arr = np.fromstring(big, sep=',', dtype=float)


In [21]:
df

,expected_weight,run_index,force_sensor_value,force_vector_1,force_vector_2,force_vector_3,robot_tcp.joint_angles_1,robot_tcp.joint_angles_2,robot_tcp.joint_angles_3,robot_tcp.joint_angles_4,...,robot_tcp.velocity_linear_1,robot_tcp.velocity_linear_2,robot_tcp.velocity_linear_3,robot_tcp.flange_1,robot_tcp.flange_2,robot_tcp.flange_3,robot_tcp.flange_4,robot_tcp.flange_5,robot_tcp.flange_6,robot_tcp.flange_7
0,0.165,1,0.115841,-0.810,-3.294,25166.0,-1.561416,-1.769448,-1.829999,-1.088762,...,-5.240363e-02,1.036457e-01,1.326491e+00,-0.169060,-0.751794,0.456477,-0.476202,0.879217,-0.014410,0.001355
1,0.165,1,0.111841,-0.828,-3.354,27135.0,-1.565000,-1.731498,-1.790243,-1.166664,...,-1.104458e-01,2.241621e-01,1.293527e+00,-0.169309,-0.751351,0.459038,-0.476191,0.879223,-0.014390,0.001346
2,0.165,1,0.109841,-0.845,-3.379,27161.0,-1.565294,-1.729290,-1.787980,-1.170969,...,-1.140317e-01,2.343790e-01,1.289273e+00,-0.169526,-0.750933,0.461644,-0.476170,0.879233,-0.014459,0.001294
3,0.165,1,0.108841,-0.869,-3.401,26560.0,-1.565294,-1.729290,-1.787980,-1.170969,...,-1.140317e-01,2.343790e-01,1.289273e+00,-0.169526,-0.750933,0.461644,-0.476170,0.879233,-0.014459,0.001294
4,0.165,1,0.107841,-0.897,-3.418,25701.0,-1.565294,-1.729290,-1.787980,-1.170969,...,-1.140317e-01,2.343790e-01,1.289273e+00,-0.169774,-0.750466,0.464228,-0.476165,0.879235,-0.014506,0.001266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90345,0.165,122,0.345844,-0.158,-0.076,29409.0,-2.677172,-1.502974,-1.141248,-2.068237,...,-2.181850e-11,-1.090531e-11,5.971910e-11,-0.597560,-0.104043,0.947141,0.456295,0.889829,-0.000009,-0.000009
90346,0.165,122,0.345844,-0.169,-0.076,29394.0,-2.677172,-1.502974,-1.141248,-2.068237,...,-2.181850e-11,-1.090531e-11,5.971910e-11,-0.597560,-0.104043,0.947141,0.456295,0.889829,-0.000009,-0.000009
90347,0.165,122,0.345844,-0.181,-0.077,29414.0,-2.677172,-1.502974,-1.141248,-2.068237,...,-2.181850e-11,-1.090531e-11,5.971910e-11,-0.597573,-0.104047,0.947136,0.456303,0.889824,-0.000005,-0.000004
90348,0.165,122,0.345844,-0.194,-0.078,29422.0,-2.677172,-1.502974,-1.141248,-2.068237,...,-2.181850e-11,-1.090531e-11,5.971910e-11,-0.597573,-0.104047,0.947136,0.456303,0.889824,-0.000005,-0.000004


In [22]:
df.to_csv('Manual_measurements_cleaned_and_structured.csv', index=False)